In [2]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13485 sha256=abdb92421106aa0886c386f8df2c892fdab612e15478827cc687ea5a0f084711
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [107]:
import urllib.request as lib
import requests
import json
import os
import re
from collections import namedtuple

In [140]:
s = lib.urlopen(GetThreadJson("https://2ch.hk/a/res/7082301.html"))
data = json.loads(s.read().decode())

In [141]:
# Исправить, когда два последовательных element - тексты
def process_comment(comment):
    comment_list = comment["comment"].split('<br>')
    Info = namedtuple('Info' , 'date num parents text')
    pairs = []
    reference_ids = []
    for i, element in enumerate(comment_list):
        text = []
        if element:
            if element.startswith('<a'):
                ref = re.findall(r'>>>(\d+)', element)
                reference_ids.append(ref[0])
            elif element[0].isalnum():
                text = element
                if not reference_ids:
                    reference_ids.append('0')
                pairs.append(Info(comment["date"], str(comment["num"]), reference_ids, text))
                reference_ids = []
    return pairs

In [142]:
example_post = data['threads'][0]['posts'][2]

In [143]:
example_post

{'banned': 0,
 'closed': 0,
 'comment': '<a href="/a/res/7082301.html#7237489" class="post-reply-link" data-thread="7082301" data-num="7237489">>>7237489</a><br><a href="/a/res/7082301.html#7237490" class="post-reply-link" data-thread="7082301" data-num="7237490">>>7237490</a><br>Да, где школьниц как кукл ломают и живьём жрут.<br><br><a href="/a/res/7082301.html#7237508" class="post-reply-link" data-thread="7082301" data-num="7237508">>>7237508</a><br>Пока что лучшее что я видел из подобных Strike blood. Ещё Шану напоминает, история в стиле &quot;как папа познакомился с мамой&quot;. Очень плавное повествование, но акшон в каждой серии. Если тебе SB зашёл - качай прямо сейчас и смотри без раздумий.',
 'date': '04/05/21 Втр 15:54:37',
 'email': '',
 'endless': 0,
 'files': [{'displayname': '[Anime365] Sous[...].jpg',
   'fullname': '[Anime365] Sousei no Onmyouji - 09 (t2397265).mp4snapshot19[...].jpg',
   'height': 895,
   'md5': '046ce1dead6d65dbf3628f889ac167f1',
   'name': '1620132878

In [145]:
example_post["comment"]

'<a href="/a/res/7082301.html#7237489" class="post-reply-link" data-thread="7082301" data-num="7237489">>>7237489</a><br><a href="/a/res/7082301.html#7237490" class="post-reply-link" data-thread="7082301" data-num="7237490">>>7237490</a><br>Да, где школьниц как кукл ломают и живьём жрут.<br><br><a href="/a/res/7082301.html#7237508" class="post-reply-link" data-thread="7082301" data-num="7237508">>>7237508</a><br>Пока что лучшее что я видел из подобных Strike blood. Ещё Шану напоминает, история в стиле &quot;как папа познакомился с мамой&quot;. Очень плавное повествование, но акшон в каждой серии. Если тебе SB зашёл - качай прямо сейчас и смотри без раздумий.'

In [134]:
process_comment(example_post)

[Info(date='26/04/21 Пнд 17:54:55', num='1188483', parents=['1188466'], text='Очевидно что откуда то из-за бугра, вопрос в цене. Если оптом брать - то в принципе косарь за месячный курс 1мг&#47;день.'),
 Info(date='26/04/21 Пнд 17:54:55', num='1188483', parents=['0'], text='Мож скинуться кто хочет? ')]

In [65]:
example_post.split('<br>'

IndexError: ignored

In [64]:
if example_post.split('<br>')[2]:
    print("h")

h


In [40]:
re.findall(r'>>>(\d+)', example_post)

['7237489', '7237490', '7237508']

In [48]:
re.findall(r'(?<=<br>).+(?=<br>)', example_post)

['<a href="/a/res/7082301.html#7237490" class="post-reply-link" data-thread="7082301" data-num="7237490">>>7237490</a><br>Да, где школьниц как кукл ломают и живьём жрут.<br><br><a href="/a/res/7082301.html#7237508" class="post-reply-link" data-thread="7082301" data-num="7237508">>>7237508</a>']

In [7]:
def main():
    thread_url = input("Thread link: ")
    s = lib.urlopen(GetThreadJson(thread_url))
    with s:
        data = json.loads(s.read().decode())
        threads_array = data['threads']
        for posts_array in threads_array:
            posts = posts_array['posts']
            for post in posts:
                
                files = files_array['files']
                for file_obj in files:
                    path = file_obj['path']
                    name = file_obj['name']
                    download("http://2ch.hk" + path, name)
                    print(path)

    print("success")

In [8]:
def download(url, output_name):
    content = requests.get(url).content
    f = open("parsed/" + str(output_name), "wb")
    f.write(content)
    f.close()

In [9]:
def GetThreadJson(url):
    convertedUrl = ""
    if not url.startswith("https://"):
        if url.startswith("http://"):
            convertedUrl = url.replace("http://", "https://")
        else:
            convertedUrl = url.replace("2ch.hk/", "https://2ch.hk/")
    else:
        convertedUrl = url

    if not url.endswith(".html"):
        if url.endswith(".json"):
            return convertedUrl
        else:
            return str(convertedUrl + ".json")
    else:
        return convertedUrl.replace(".html", ".json")

In [10]:
if __name__ == "__main__":
    if not os.path.exists("parsed"):
        os.makedirs("parsed")
    
    main()

Thread link: https://2ch.hk/a/res/7082301.html
/a/src/7082301/16093369516980.jpg
/a/src/7082301/16201328783340.jpg
/a/src/7082301/16201343796780.png
/a/src/7082301/16201346198820.jpg
/a/src/7082301/16201346461920.jpg
/a/src/7082301/16201346461931.jpg
/a/src/7082301/16201346461932.jpg


KeyboardInterrupt: ignored

In [5]:
{"Board":"biz",
 "BoardInfo":"\u003cfont color=\"red\"\u003eНе нужны: поиск работы по найму, пирамиды и ваши реферальные ссылки, волшебные кошельки, выпрашивание денег у анонов и хайп-проекты.\u003cbr\u003eОбсуждение работы и карьеры в \u003ca href=\"https://2ch.hk/wrk/\"\u003e/wrk/\u003c/a\u003e Вопросы юридического характера можно решить в \u003ca href=\"https://2ch.hk/law/\"\u003e/law/\u003c/a\u003e Криптовалюты в \u003ca href=\"https://2ch.hk/cc/\"\u003e/сс/\u003c/a\u003e\u003c/font\u003e",
 "BoardInfoOuter":"бизнес, сириуз бизнес",
 "BoardName":"Бизнес",
 "advert_bottom_image":"/banners/CExQeE4tdixQmH1q.jpg",
 "advert_bottom_link":"/banners/CExQeE4tdixQmH1q/",
 "advert_mobile_image":"/banners/0LyqzyvkiidLWqqm.jpg",
 "advert_mobile_link":"/banners/0LyqzyvkiidLWqqm/",
 "advert_top_image":"/banners/kVoisYw85yR6eb16.jpg",
 "advert_top_link":"/banners/kVoisYw85yR6eb16/",
 "board_banner_image":"/ololo/d_4.png",
 "board_banner_link":"d",
 "bump_limit":500,
 "current_thread":"1519294",
 "default_name":"Аноним",
 "enable_dices":0,
 "enable_flags":0,
 "enable_icons":0,
 "enable_images":1,
 "enable_likes":0,
 "enable_names":1,
 "enable_oekaki":0,
 "enable_posting":1,
 "enable_sage":1,
 "enable_shield":0,
 "enable_subject":1,"enable_thread_tags":0,"enable_trips":1,"enable_video":1,"files_count":1,"is_board":0,"is_closed":0,"is_index":0,"max_comment":15000,"max_files_size":40960,"max_num":1524248,"news_abu":[{"date":"02/12/16","num":54946,"subject":"Конкурс визуальных новелл доски /ruvn/","views":146},{"date":"15/11/16","num":53612,"subject":"**НОВЫЙ ФУНКЦИОНАЛ** - Стикеры","views":38863},{"date":"09/10/16","num":52084,"subject":"Открыта доска /int/ - International, давайте расскажем о ней!","views":34216}],"posts_count":5,"thread_first_image":"/biz/src/1519294/16200360096610.jpg","threads":[{"posts":[{"banned":0,"closed":0,"comment":"Что можно открыть в подвале в городе без вложения денег в ремонт\u003cbr\u003e\u003cbr\u003eЕсть подвальное помещение, раньше использовалось под офис, сейчас в убитом состояние. Хочу хоть как-то на нем зарабатывать, но в голову приходит только какое-нибудь изготовление ключей. \u003cbr\u003e\u003cbr\u003eАнончики направьте пожалуйста, в какую сторону двигаться","date":"03/05/21 Пнд 13:00:08","email":"","endless":0,"files":[{"displayname":"IMG202102071056[...].jpg","fullname":"IMG20210207105628584.jpg","height":777,"md5":"f3c5bf34e01b2dfff7e1d19b805c4693","name":"16200360096610.jpg","nsfw":0,"path":"/biz/src/1519294/16200360096610.jpg","size":75,"thumbnail":"/biz/thumb/1519294/16200360096610s.jpg","tn_height":250,"tn_width":241,"type":1,"width":750}],"lasthit":1620381126,"name":"Аноним","num":1519294,"number":1,"op":0,"parent":"0","sticky":0,"subject":"Что можно открыть в подвале в городе без вложения","tags":"","timestamp":1620036008,"trip":""},{"banned":0,"closed":0,"comment":"Сколько квадратных метров? Отопление свет дорогие? Какая климатическая зона? Сколько этажная застройка вокруг? Ну там одноэтажные домики или башни по 40 этажей?","date":"03/05/21 Пнд 14:54:45","email":"","endless":0,"files":[],"lasthit":1620381126,"name":"Аноним","num":1519315,"number":2,"op":0,"parent":"1519294","sticky":0,"subject":"","timestamp":1620042885,"trip":""},{"banned":0,"closed":0,"comment":"\u003ca href=\"/biz/res/1519294.html#1519315\" class=\"post-reply-link\" data-thread=\"1519294\" data-num=\"1519315\"\u003e\u003e\u003e1519315\u003c/a\u003e\u003cbr\u003e70 квадратов, отопление и свет хз. ЦФО, а если точнее, то Калуга.\u003cbr\u003eесть и 5-этажки, есть и многоэтажные дома. совсем рядом почти достроили несколько многоэтажных домов.","date":"03/05/21 Пнд 15:23:14","email":"","endless":0,"files":[],"lasthit":1620381126,"name":"Аноним","num":1519329,"number":3,"op":0,"parent":"1519294","sticky":0,"subject":"","timestamp":1620044594,"trip":""},{"banned":0,"closed":0,"comment":"\u003ca href=\"/biz/res/1519294.html#1519294\" class=\"post-reply-link\" data-thread=\"1519294\" data-num=\"1519294\"\u003e\u003e\u003e1519294 (OP)\u003c/a\u003e\u003cbr\u003eКлуб настольных игр, днд там всякое, мтг, Ваха. Разруха только добавит атмосферы.","date":"03/05/21 Пнд 15:28:07","email":"","endless":0,"files":[],"lasthit":1620381126,"name":"Аноним","num":1519335,"number":4,"op":0,"parent":"1519294","sticky":0,"subject":"","timestamp":1620044887,"trip":""},{"banned":0,"closed":0,"comment":"\u003ca href=\"/biz/res/1519294.html#1519294\" class=\"post-reply-link\" data-thread=\"1519294\" data-num=\"1519294\"\u003e\u003e\u003e1519294 (OP)\u003c/a\u003e\u003cbr\u003eтреш вебкам студио.\u003cbr\u003e\u003cbr\u003eОхуеть, как же все верят что легко и без вложений можно бабки рубить. Во-первых, уебал помещение - делай ремонт. В помойку ни один клиент не пойдет.\u003cbr\u003eВо-вторых, если хочешь бизнес делай любую услугу, в которой разбираешься. Нет понимания? Пиздуй работать и учиться ремеслу.\u003cbr\u003eСамый рабочий вариант зарабатывать для таких Васянов с прилипшей недвигой как ты только сдавать в аренду.\u003cbr\u003eЕсли есть большое желание все-таки делать бизнес, иди ногами по дворам и смотри что в похожих условиях работает. То примеряй на себя и после запускай.","date":"07/05/21 Птн 06:35:00","email":"","endless":0,"files":[],"lasthit":1620381126,"name":"Аноним","num":1523970,"number":5,"op":0,"parent":"1519294","sticky":0,"subject":"","timestamp":1620358500,"trip":""},{"banned":0,"closed":0,"comment":"\u003ca href=\"/biz/res/1519294.html#1519294\" class=\"post-reply-link\" data-thread=\"1519294\" data-num=\"1519294\"\u003e\u003e\u003e1519294 (OP)\u003c/a\u003e\u003cbr\u003e\u003cspan class=\"unkfunc\"\u003e&gt; Есть подвальное помещение, раньше использовалось под офис, сейчас в убитом состояние. Хочу хоть как-то на нем зарабатывать, но в голову приходит только какое-нибудь изготовление ключей.\u003c/span\u003e\u003cbr\u003eСдай его, с условием ремонта в счет аренды. Через годик-другой, когда арендатор раскрутится, выселяешь его из помещения и открываешь по горячим следам анало-гичный бизнес.","date":"07/05/21 Птн 12:52:06","email":"","endless":0,"files":[],"lasthit":1620381126,"name":"Аноним","num":1524248,"number":6,"op":0,"parent":"1519294","sticky":0,"subject":"","timestamp":1620381126,"trip":""}]}],"title":"Что можно открыть в подвале в городе без вложения","top":[{"board":"2d","info":"Щитпостинг, обсуждение вайфу, аватарки и прочее. Анимешный /b/, постинг 3d не приветствуется.","name":"Аниме/Беседка"},{"board":"wwe","info":"Доска, посвященная рестлингу: WWE, NXT, TNA, ROH, NJPW, Lucha Underground и многое другое","name":"WorldWide Wrestling Universe"},{"board":"ch","info":"Уютные чатики и конфочки","name":"Чатики и конфочки"},{"board":"int","info":"This board focuses on international friendship and cultural exchange.","name":"International"},{"board":"ruvn","info":"","name":"Российские визуальные новеллы"},{"board":"math","info":"Доска о модулях над кольцами, пучках на многообразиях и гомологиях с когомологиями.","name":"Математика"}],"unique_posters":"5"}

{'Board': 'biz',
 'BoardInfo': '<font color="red">Не нужны: поиск работы по найму, пирамиды и ваши реферальные ссылки, волшебные кошельки, выпрашивание денег у анонов и хайп-проекты.<br>Обсуждение работы и карьеры в <a href="https://2ch.hk/wrk/">/wrk/</a> Вопросы юридического характера можно решить в <a href="https://2ch.hk/law/">/law/</a> Криптовалюты в <a href="https://2ch.hk/cc/">/сс/</a></font>',
 'BoardInfoOuter': 'бизнес, сириуз бизнес',
 'BoardName': 'Бизнес',
 'advert_bottom_image': '/banners/CExQeE4tdixQmH1q.jpg',
 'advert_bottom_link': '/banners/CExQeE4tdixQmH1q/',
 'advert_mobile_image': '/banners/0LyqzyvkiidLWqqm.jpg',
 'advert_mobile_link': '/banners/0LyqzyvkiidLWqqm/',
 'advert_top_image': '/banners/kVoisYw85yR6eb16.jpg',
 'advert_top_link': '/banners/kVoisYw85yR6eb16/',
 'board_banner_image': '/ololo/d_4.png',
 'board_banner_link': 'd',
 'bump_limit': 500,
 'current_thread': '1519294',
 'default_name': 'Аноним',
 'enable_dices': 0,
 'enable_flags': 0,
 'enable_icons': 0,